In [ ]:
import openpyxl
# 'https://www.mext.go.jp/content/20230420_mxt_daigakuc01_000230747_01.xlsx'
# エクセルファイルの取り込み
# read_only=True, data_only=True にしないと遅い（その代わり、ws.row_dimensions[i].hidden==True は使えない）

# ファイルのパスを指定してエクセルを読み込でシート名を返す
# read_only=True, data_only=True としないと時間が掛かりすぎて終わらない
def load_excel(path):
    wb = openpyxl.load_workbook(filename=path,
                               read_only=True,
                               data_only=True)
    # ws = wb.worksheets[0]
    # シートを取得
    sheets = wb.sheetnames
    # print(sheets)
    return wb, sheets

# シート名からシートを読み込む
def load_sheet(wb, sheet_name):
    # ws = wb.worksheets[sheet_name]
    ws = wb[sheet_name]
    return ws

'''
wb, sheets = load_excel(koritsu_path)
print(sheets[0])
ws = load_sheet(wb, sheets[3])
print(ws["B1:B2"])
'''



In [65]:
# シートごとの、項目のセルの範囲
cell_range_dict = {"univ_name" :"B1:B1", "univ_codekenkyuka" :"B5:B5", "president" :"C5:C5", 
                   "univ_post_code" :"J5:J5", "Univ_address" :"L5:L5", "campus" :"B13:B62", 
                   "campus_post" :"G13:G62", "campus_address" :"I13:I62", "dept_inst" :"B66:B115", 
                   "dept_inst_post" :"E66:E115", 
                   "dept_inst_address" :"G66:G115", "gakubu" :"B223:I322", "kenkyuka" : "B381:I480"}

# セル範囲を引数にして、リストを返す
def get_cell(cell_range, univ, wb):
    values_list = []
    ws = load_sheet(wb, univ)
    for i, row in enumerate(ws[cell_range]):   # (ws["A1:I481"]):
        values = []
        for col in row:
            # print(col.value)
            # if not col.value == None: # 下ので消すからよい
               values.append(col.value)
        # リスト内包表記で is not None を使ってリストからNoneを削除
        values_filtered =  [x for x in values if x is not None] # filter(lambda a:a is not None, values)
        if list(values_filtered) :  # listがカラでなければ、リストに加える
            values_list.append(list(values_filtered))
    if len(list(values_list)) == 1:
         value_list2 = [ flatten for inner in values_list for flatten in inner ]
         value_list2 = ''.join(value_list2)
         # print(value_list2)
    else:
         value_list2 = list(values_list)
    return value_list2 # list(values_list)  # ['文学部', '人文科学科', '北海道', '札幌市'], ['教育学部', '教育学科', '北海道', '札幌市'],...

# test
kokuritsu_path = r'C:\Users\onowa\Downloads\大学一覧\20230420_mxt_daigakuc01_000230747_01.xlsx'
koritsu_path = r"C:\Users\onowa\Downloads\大学一覧\20230420_mxt_daigakuc01_000230747_02.xlsx"
wb, sheets = load_excel(kokuritsu_path)
sheet_name = sheets[0]
print(sheets)
cell_range = "B1:B1" # "A381:I480" # "A381:I480"  # 'A381:I480' # ['小樽商科大学', '商学研究科', None, '現代商学専攻']
print(get_cell(cell_range, sheet_name, wb))

['北海道大学', '北海道教育大学', '室蘭工業大学', '小樽商科大学', '帯広畜産大学', '旭川医科大学', '北見工業大学', '弘前大学', '岩手大学', '東北大学', '宮城教育大学', '秋田大学', '山形大学', '福島大学', '茨城大学', '筑波大学', '筑波技術大学', '宇都宮大学', '群馬大学', '埼玉大学', '千葉大学', '東京大学', '東京医科歯科大学', '東京外国語大学', '東京学芸大学', '東京農工大学', '東京芸術大学', '東京工業大学', '東京海洋大学', 'お茶の水女子大学', '電気通信大学', '一橋大学', '横浜国立大学', '新潟大学', '長岡技術科学大学', '上越教育大学', '富山大学', '金沢大学', '福井大学', '山梨大学', '信州大学', '岐阜大学', '静岡大学', '浜松医科大学', '名古屋大学', '愛知教育大学', '名古屋工業大学', '豊橋技術科学大学', '三重大学', '滋賀大学', '滋賀医科大学', '京都大学', '京都教育大学', '京都工芸繊維大学', '大阪大学', '大阪教育大学', '兵庫教育大学', '神戸大学', '奈良教育大学', '奈良女子大学', '和歌山大学', '鳥取大学', '島根大学', '岡山大学', '広島大学', '山口大学', '徳島大学', '鳴門教育大学', '香川大学', '愛媛大学', '高知大学', '福岡教育大学', '九州大学', '九州工業大学', '佐賀大学', '長崎大学', '熊本大学', '大分大学', '宮崎大学', '鹿児島大学', '鹿屋体育大学', '琉球大学', '政策研究大学院大学', '総合研究大学院大学', '北陸先端科学技術大学院大学', '奈良先端科学技術大学院大学']
国立 北海道大学（Hokkaido University）


In [76]:
import itertools
import numpy as np

# 多次元リストのフラット化
def ndim_list(ndimension_list):
    return list(itertools.chain.from_iterable(ndimension_list))

# 大学名（=sheeet名）を引数に、項目の辞書を返す
def get_item( univ, wb ):
    item_list = []    
    for item_name in cell_range_dict:
        # セル範囲の辞書（項目名）を引数にして、項目名：項目 の辞書を返す
        cell_range_name = get_cell(cell_range_dict[item_name], univ, wb)
        # item_name_dict = {item_name : cell_range_name}
        # item_name_dict = {item_name : list(itertools.chain.from_iterable(cell_range_name))}
        if np.ndim(cell_range_name) > 1:
            # print(np.ndim(cell_range_name)) # 配列の(numpy)次元数を調べる
            # item_name_dict = {item_name : ndim_list(cell_range_name)}
            item_name_dict = {item_name : str(cell_range_name)}
        else:
            item_name_dict = {item_name : cell_range_name}
        item_list.append(item_name_dict)
    flatten_dict = {k: v for child_dict in item_list for k, v in child_dict.items()} # 辞書のリストを展開
    return flatten_dict

# get_item(sheets3[0], wb2) # （sheet_name)
get_item(sheets[0], wb) 

{'univ_name': '国立 北海道大学（Hokkaido University）',
 'univ_codekenkyuka': 'F101110100010',
 'president': '寳金 清博',
 'univ_post_code': '060－0808',
 'Univ_address': '北海道札幌市北区北8条西5丁目',
 'campus': "[['法学研究科・法学部'], ['水産科学院・水産科学研究院・水産学部（函館キャンパス）'], ['環境科学院・地球環境科学研究院'], ['理学院・理学研究院・理学部'], ['薬学研究院・薬学部'], ['農学院・農学研究院・農学部'], ['生命科学院'], ['先端生命科学研究院'], ['教育学院・教育学研究院・教育学部'], ['国際広報メディア・観光学院・メディア・コミュニケーション研究院'], ['保健科学院・保健科学研究院・医学部（保健学科）'], ['工学院・工学研究院・工学部'], ['総合化学院'], ['経済学院・経済学研究院・経済学部'], ['医学院・医学研究院・医学部（医学科）'], ['歯学院・歯学研究院・歯学部'], ['獣医学院・獣医学研究院・獣医学部'], ['医理工学院'], ['国際感染症学院'], ['国際食資源学院'], ['文学院・文学研究院・文学部'], ['情報科学院・情報科学研究院'], ['公共政策学教育部・公共政策学連携研究部']]",
 'campus_post': "[['060－0809'], ['041－8611'], ['060－0810'], ['060－0810'], ['060－0812'], ['060－8589'], ['060－0810'], ['060－0810'], ['060－0811'], ['060－0817'], ['060－0812'], ['060－8628'], ['060－8628'], ['060－0809'], ['060－8638'], ['060－8586'], ['060－0818'], ['060－8638'], ['060－0818'], ['060－0809'], ['060－0810'], ['060－0814'], ['060－0809']]",
 'campus_addre

In [30]:
# 指定したディレクトリのファイルをリストで取得
from pathlib import Path

# Pathオブジェクトを生成
p = Path(r"C:\Users\onowa\Downloads\大学一覧")

# dir_A直下のファイルとディレクトリを取得
# Path.glob(pattern)はジェネレータを返す。結果を明示するためlist化しているが、普段は不要。
files = list(p.glob("*"))
# 再帰的な検索  # list(p.glob("**/*"))
files[0]



WindowsPath('C:/Users/onowa/Downloads/大学一覧/20230420_mxt_daigakuc01_000230747_01.xlsx')

In [52]:
# kokuritsu_path = r'C:\Users\onowa\Downloads\大学一覧\20230420_mxt_daigakuc01_000230747_01.xlsx'
# koritsu_path = r"C:\Users\onowa\Downloads\大学一覧\20230420_mxt_daigakuc01_000230747_02.xlsx"
# wb2, sheets3 = load_excel(files[3])  # koritsu_path

def universities_dict(wb2, sheets3):
    # 大学名のリストをループして、辞書に追加していく
    university = {}
    print(sheets3[0])
    for sheet_name in sheets3:
        Univ_item2 = {sheet_name:get_item(sheet_name, wb2)}
        university.update(Univ_item2)
    return university

university_all = {}
for file_name in files:
    wb2, sheets3 = load_excel(file_name)
    university_all.update(universities_dict(wb2, sheets3))
print(university_all)
len(university_all)
# 14.0s

北海道大学
札幌医科大学
札幌大学
日本医療科学大学
清泉女子大学
東京純心大学
山梨学院大学
びわこ成蹊スポーツ大学
宝塚大学
放送大学
大学共同利用機関法人
私立通信教育課程（大学）
{'北海道大学': {'univ_name': '国立 北海道大学（Hokkaido University）', 'univ_codekenkyuka': 'F101110100010', 'president': '寳金 清博', 'univ_post_code': '060－0808', 'Univ_address': '北海道札幌市北区北8条西5丁目', 'campus': [['法学研究科・法学部'], ['水産科学院・水産科学研究院・水産学部（函館キャンパス）'], ['環境科学院・地球環境科学研究院'], ['理学院・理学研究院・理学部'], ['薬学研究院・薬学部'], ['農学院・農学研究院・農学部'], ['生命科学院'], ['先端生命科学研究院'], ['教育学院・教育学研究院・教育学部'], ['国際広報メディア・観光学院・メディア・コミュニケーション研究院'], ['保健科学院・保健科学研究院・医学部（保健学科）'], ['工学院・工学研究院・工学部'], ['総合化学院'], ['経済学院・経済学研究院・経済学部'], ['医学院・医学研究院・医学部（医学科）'], ['歯学院・歯学研究院・歯学部'], ['獣医学院・獣医学研究院・獣医学部'], ['医理工学院'], ['国際感染症学院'], ['国際食資源学院'], ['文学院・文学研究院・文学部'], ['情報科学院・情報科学研究院'], ['公共政策学教育部・公共政策学連携研究部']], 'campus_post': [['060－0809'], ['041－8611'], ['060－0810'], ['060－0810'], ['060－0812'], ['060－8589'], ['060－0810'], ['060－0810'], ['060－0811'], ['060－0817'], ['060－0812'], ['060－8628'], ['060－8628'], ['060－0809'], ['060－8638'], ['060－8586'], ['060－0818'], ['060

758

In [47]:
print(sheets)
print(wb2)
print(files[4])

['日本医療科学大学', '日本薬科大学', '武蔵野学院大学', '浦和大学', 'ものつくり大学', '共栄大学', '埼玉学園大学', '尚美学園大学', '人間総合科学大学', '西武文理大学', '十文字学園女子大学', '平成国際大学', '東京国際大学', '城西大学', '東邦音楽大学', '獨協大学', '日本工業大学', '明海大学', '埼玉医科大学', '埼玉工業大学', '駿河台大学', '聖学院大学', '女子栄養大学', '東都大学', '日本保健医療大学', '千葉科学大学', '愛国学園大学', '開智国際大学', '清和大学', '城西国際大学', '淑徳大学', '敬愛大学', '千葉工業大学', '千葉商科大学', '中央学院大学', '麗澤大学', '和洋女子大学', '国際武道大学', '神田外語大学', '千葉経済大学', '秀明大学', '川村学園女子大学', '東京情報大学', '東京基督教大学', '聖徳大学', '江戸川大学', '了徳寺大学', '植草学園大学', '三育学院大学', '亀田医療大学', '東京福祉大学', '跡見学園女子大学', '文教大学', '文京学院大学', '目白大学', '東京成徳大学', '東洋学園大学', '帝京平成大学', '青山学院大学', '上野学園大学', '大妻女子大学', '学習院大学', '北里大学', '共立女子大学', '慶應義塾大学', '工学院大学', '國學院大學', '国士舘大学', '駒澤大学', '実践女子大学', '芝浦工業大学', '順天堂大学', '上智大学', '昭和大学', '昭和女子大学', '昭和薬科大学', '女子美術大学', '杉野服飾大学', '成城大学', '聖心女子大学']
C:\Users\onowa\Downloads\大学一覧\20230420_mxt_daigakuc01_000230747_03-3.xlsx


In [55]:
# データフレームに変換し、エクセルに出力
import pandas as pd
df=pd.DataFrame(university_all)
univall_dfT = df.T
# データフレームをpickleに保存
univall_dfT.to_pickle('data/university_all.zip')
with pd.ExcelWriter('university_all.xlsx') as writer:  # , mode='a'
    univall_dfT.to_excel(writer)   # , sheet_name='new_sheet1')

In [58]:
# print(univall_dfT)
univall_dfT.columns

Index(['univ_name', 'univ_codekenkyuka', 'president', 'univ_post_code',
       'Univ_address', 'campus', 'campus_post', 'campus_address', 'dept_inst',
       'dept_inst_post', 'dept_inst_address', 'gakubu', 'kenkyuka'],
      dtype='object')

In [59]:
univall_dfT['kenkyuka']

北海道大学              [[法学研究科, 法学政治学専攻, 北海道, 札幌市], [法学研究科, 法律実務専攻, 北...
北海道教育大学            [[教育学研究科, 学校教育専攻, 北海道, 札幌市], [教育学研究科, 教科教育専攻, ...
室蘭工業大学             [[工学研究科, 環境創生工学系専攻, 北海道, 室蘭市], [工学研究科, 生産システム工...
小樽商科大学             [[商学研究科, 現代商学専攻, 北海道, 小樽市], [商学研究科, アントレプレナーシッ...
帯広畜産大学             [[畜産学研究科, 畜産科学専攻, 北海道, 帯広市], [畜産学研究科, 獣医学専攻, 北...
                                         ...                        
大学別科                                                              []
国立大学附置研究所                                                         []
国立大学附置研究所附属施設                                                     []
共同実施制度を利用している大学                                                   []
連合大学院を置く大学                                                        []
Name: kenkyuka, Length: 758, dtype: object